# 🔍 Exploring Transformer Models (Lower-Level API)

This notebook dives into the **lower-level API of Transformers**, focusing on the underlying models that wrap PyTorch code to implement transformer architectures.

> 💡 **Runtime Note:** This notebook is optimized to run on a **low-cost or free T4 runtime** (such as Google Colab's free tier).

## 📦 Models Explored

This notebook explores five cutting-edge transformer models from different organizations:

- **LLaMA 3.1** by Meta  
- **Phi-3** by Microsoft  
- **Gemma** by Google  
- **Mixtral** by Mistral  
- **Qwen** by Alibaba Cloud  

Each model is examined through its lower-level PyTorch-based implementation using the Hugging Face `transformers` library, offering insights into their structure and customization potential.


Important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Kernel menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [ ]:
# ------------------------------------ Packages ----------------------------------
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [ ]:
# ------------------------------------ Imports ----------------------------------
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc

# Generate Hugging Face Token
https://huggingface.co

In [ ]:
# ------------------------------------ Configure Hugging Face Token ----------------------------------
# Retrieve stored API key from Colab's secure userdata store
hf_token = userdata.get('HF_TOKEN')

if hf_token:
    print(f"Hugging Face Token exists and begins {hf_token[:10]}")
else:
  print("Hugging Face Token not set")

# ------------------------------------ Connect to Hugging Face ----------------------------------
login(hf_token, add_to_git_credential=True)

# Request Access to HuggingFace Model:
# https://huggingface.co/black-forest-labs/FLUX.1-schnell

In [ ]:
# Initialize Model Variables

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # If this doesn't fit it your GPU memory, try others from the hub
QWEN2 = "Qwen/Qwen2-7B-Instruct" # exercise for you


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

Here's a Markdown explanation of the Python function that sets up a BitsAndBytesConfig for 4-bit quantization:

## Explanation of `BitsAndBytesConfig` for 4-bit Quantization

The following Python code configures 4-bit quantization using the `BitsAndBytesConfig` class, typically used with Hugging Face Transformers and the `bitsandbytes` library to reduce memory usage and improve inference speed:

```python
from transformers import BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

Parameter Breakdown

    load_in_4bit=True
    Enables loading the model in 4-bit precision, which drastically reduces memory usage and accelerates inference.

    bnb_4bit_use_double_quant=True
    Activates double quantization, which means that weights are first quantized to an intermediate quant level before being quantized again to 4-bit. This typically helps retain better model performance.

    bnb_4bit_compute_dtype=torch.bfloat16
    Sets the compute datatype for the quantized model. Using bfloat16 (Brain Float 16) enables faster computation on modern hardware with minimal loss in precision.

    bnb_4bit_quant_type="nf4"
    Specifies the quantization scheme.

        "nf4" stands for Normal Float 4, a quantization type developed for better range and accuracy compared to older 4-bit formats like fp4.

Use Case

This configuration is ideal for:

    Running large language models on limited hardware (e.g., consumer GPUs).

    Accelerating inference while maintaining reasonable accuracy.

    Deployments where memory footprint is a concern.

Note: This setup requires the bitsandbytes library and is often used with models from the Hugging Face Transformers ecosystem.

In [ ]:
# ------------------------------------ Quantization ----------------------------------
# Quantization Config - this allows us to load the model into memory and use less memory.
# We do this by using less precision (replacing 32-bit numbers into 8-bit numbers).  This
# is known as quantization and only makes it slightly less precise.

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
# ------------------------------------ Tokenizer ----------------------------------
# Load the tokenizer associated with the pretrained model.  LLAMA should be a string path ("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer = AutoTokenizer.from_pretrained(LLAMA)

# Sets the padding token to be the same as the end-of-sequence (EOS) token.
# This is necessary for models that don’t have a separate padding token defined by default.
# This prevents errors during batch processing or when padding sequences to equal length.
tokenizer.pad_token = tokenizer.eos_token

# Applies a chat-specific formatting template to the list of messages.
#   *messages should be a list of dictionaries (e.g., {"role": "user", "content": "Hello"}).
#   *Converts the messages into a single formatted string (list of tokens) as expected by chat-tuned models.
#   *return_tensors="pt" returns PyTorch tensors for model input.
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
# .to("cuda") - Moves the resulting tensor to the GPU for faster inference.


In [ ]:
# ------------------------------------ The Model ----------------------------------
# Load a pretrained LLaMA model ("meta-llama/Meta-Llama-3.1-8B-Instruct") for text generation
#   *Loads using memory-efficient 4-bit quantization
#   *device_map="auto" - This automatically distributes the model across available devices
#    (e.g., multiple GPUs or CPU + GPU), if your Colab notebook is configured to use GPU.
#   *It takes tokens from the past to predict future tokens
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

In [ ]:
# ------------------------------------ Memory Footprint ----------------------------------
# This code measures and prints the model's memory usage in megabytes (MB).
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

**Output:**  
Memory footprint: 5,591.5 MB


## Looking under the hood at the Transformer model

The next cell prints the HuggingFace `model` object for Llama.

This model object is a Neural Network, implemented with the Python framework PyTorch. The Neural Network uses the architecture invented by Google scientists in 2017: the Transformer architecture.

While we're not going to go deep into the theory, this is an opportunity to get some intuition for what the Transformer actually is.

If you're completely new to Neural Networks, check out my [YouTube intro playlist](https://www.youtube.com/playlist?list=PLWHe-9GP9SMMdl6SLaovUQF2abiLGbMjs) for the foundations.

Now take a look at the layers of the Neural Network that get printed in the next cell. Look out for this:

- It consists of layers
- There's something called "embedding" - this takes tokens and turns them into 4,096 dimensional vectors. We'll learn more about this in Week 5.
- There are then 32 sets of groups of layers called "Decoder layers". Each Decoder layer contains three types of layer: (a) self-attention layers (b) multi-layer perceptron (MLP) layers (c) batch norm layers.
- There is an LM Head layer at the end; this produces the output

Notice the mention that the model has been quantized to 4 bits.

It's not required to go any deeper into the theory at this point, but if you'd like to, I've asked our mutual friend to take this printout and make a tutorial to walk through each layer. This also looks at the dimensions at each point. If you're interested, work through this tutorial after running the next cell:

https://chatgpt.com/canvas/shared/680cbea6de688191a20f350a2293c76b

In [ ]:
# Execute this cell and look at what gets printed; investigate the layers

model

**Output:**  
  
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)

### And if you want to go even deeper into Transformers

In addition to looking at each of the layers in the model, you can actually look at the HuggingFace code that implements Llama using PyTorch.

Here is the HuggingFace Transformers repo:  
https://github.com/huggingface/transformers

And within this, here is the code for Llama 4:  
https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama4/modeling_llama4.py

Obviously it's not neceesary at all to get into this detail - the job of an AI engineer is to select, optimize, fine-tune and apply LLMs rather than to code a transformer in PyTorch. OpenAI, Meta and other frontier labs spent millions building and training these models. But it's a fascinating rabbit hole if you're interested!

In [ ]:
# ------------------------------------ Run The Model ----------------------------------
outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

**Output:**

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the logistic regression model go to therapy?

Because it was struggling to classify its emotions.<|eot_id|>


In [ ]:
# ------------------------------------ Clean Up Memory ----------------------------------
# Delete variables from the current Python namespace
del model, inputs, tokenizer, outputs
# Manually call garbage collector to clean up unreferenced memory
gc.collect()
# Tell PyTorch to release cached GPU memory that is no longer being used
torch.cuda.empty_cache()

## A couple of quick notes on the next block of code:

I'm using a HuggingFace utility called TextStreamer so that results stream back.
To stream results, we simply replace:  
`outputs = model.generate(inputs, max_new_tokens=80)`  
With:  
`streamer = TextStreamer(tokenizer)`  
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

Also I've added the argument `add_generation_prompt=True` to my call to create the Chat template. This ensures that Phi generates a response to the question, instead of just predicting how the user prompt continues. Try experimenting with setting this to False to see what happens. You can read about this argument here:

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts

Thank you to student Piotr B for raising the issue!

In [ ]:
# ------------------------------------ Wrap Everything Into a Function ----------------------------------
def generate(model, messages):
    # Load the tokenizer for the specified model
    tokenizer = AutoTokenizer.from_pretrained(model)

    # Set the tokenizer's padding token to be the same as the end-of-sequence token
    # This avoids errors when padding sequences for generation
    tokenizer.pad_token = tokenizer.eos_token

    # Format the conversation messages using the chat template and convert to tensor on GPU
    # 'add_generation_prompt=True' prepares the input for text generation continuation
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to("cuda")

    # Set up a streaming output handler for real-time token decoding (optional, useful for live output)
    streamer = TextStreamer(tokenizer)

    # Load the model onto the appropriate device (GPU/CPU) with quantization if specified
    model = AutoModelForCausalLM.from_pretrained(
        model,
        device_map="auto",
        quantization_config=quant_config
    )

    # Generate up to 80 new tokens based on the input and stream them live
    outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)

    # Clean up: delete objects to free memory (especially GPU memory)
    del model, inputs, tokenizer, outputs, streamer
    gc.collect()                 # Run Python garbage collector
    torch.cuda.empty_cache()    # Free unused GPU memory cache


In [ ]:
# ------------------------------------ Call PHI 3 ----------------------------------
generate(PHI3, messages)

**Output:**  
Loading checkpoint shards: 100%
 2/2 [00:37<00:00, 17.80s/it]

<|user|> Tell a light-hearted joke for a room of Data Scientists<|end|><|assistant|> Why did the data scientist break up with the algorithm?

Because it was always calculating the wrong variables in their relationship!<|end|>


## Accessing Gemma from Google
Google requires you to accept their terms in HuggingFace before you use Gemma.

Visit their model page at this link and confirm you're OK with their terms, so that you're granted access.

https://huggingface.co/google/gemma-2-2b-it

In [ ]:
# ------------------------------------ Call GEMMA2 ----------------------------------
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(GEMMA2, messages)

**Output:**  
<bos><start_of_turn>user  
Tell a light-hearted joke for a room of Data Scientists<end_of_turn>  
<start_of_turn>model  
Why did the data scientist break up with the statistician?   
  
Because they had too many differences! 😂   
  
---
  
Let me know if you'd like to hear another joke! 😊   
<end_of_turn>

**Request Access to MIXTRAL:**  
https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

In [ ]:
# ------------------------------------ Call MIXTRAL ----------------------------------
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(MIXTRAL, messages)



Request Access to QWEN2:
https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1


In [ ]:
# ------------------------------------ Call QWEN2 ----------------------------------
# Use 8-bit instead of 4-bit to free up GPU for the model
quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(QWEN2, messages)